In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### detecting number of components in tissue area and seperating cells based on the size

In [3]:
img = cv2.imread(r'images\black_bubbles_3.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [4]:
# binarize the image
ret, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [7]:
# find connected components
connectivity = 4
num_cell_type1 = 0
num_cell_type2 = 0
num_cell_type3 = 0

nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(bw, connectivity, cv2.CV_32S)
sizes = stats[1:, -1]; nb_components = nb_components - 1
print("# of cells:",nb_components)
min_size = 500 #threshhold value for objects in scene
img2 = np.zeros((img.shape), np.uint8)
img_label = np.zeros((img.shape[0],img.shape[1],1), np.uint8)

for i in range(0, nb_components):
    if sizes[i] <= 150: #to identify your objects
        color = [255,0,0]
        value = 2
        num_cell_type2+=1
    elif sizes[i] >150 and sizes[i]<350:
        color = [0,255,0]
        value = 1
        num_cell_type1+=1
    elif sizes[i] >=350 and sizes[i]<500:
        color = [0,0,255]
        value = 3
        num_cell_type3+=1
    else:#background
        color = [0,0,0]
        value = 0
    # draw the bounding rectangele around each object
    cv2.rectangle(img2, (stats[i][0],stats[i][1]),(stats[i][0]+stats[i][2],stats[i][1]+stats[i][3]), color, 2)
    img2[output == i + 1] = color
    img_label[output == i + 1] = value

print('num_cell_type1 :%d,\nnum_cell_type2 : %d,\nnum_cell_type3 :%d'%(num_cell_type1,num_cell_type2,num_cell_type3))

# of cells: 170
num_cell_type1 :52,
num_cell_type2 : 53,
num_cell_type3 :60


### saving groundtruth images to train neural network

In [8]:
plt.imsave("black_bubbles_3_gt_rec.png",img2)
np.save("black_bubbles_3_gt.npy",img_label)